In [1]:
import tensorflow as tf
from tensorflow import keras
import datasets

In [ ]:
# from keras.layers import Dropout
# from keras import regularizers, optimizers
# from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D, TimeDistributed
# from keras.models import Model, load_model
# from keras.layers.core import Reshape
#
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed
# from keras.utils import plot_model
#
# from keras.layers import Input, LSTM, RepeatVector
# from keras.models import Model

In [2]:
### Paths

ptb_char_train_path = "datasets\ptb\ptb.char.train.txt"
ptb_char_valid_path = "datasets\ptb\ptb.char.valid.txt"
ptb_word_train_path = "datasets\ptb\ptb.train.txt"
ptb_word_valid_path = "datasets\ptb\ptb.valid.txt"
ptb_word_test_path = "datasets\ptb\ptb.test.txt"

wikitext2_train_path = "datasets\wikitext-2-v1\wikitext-2\wiki.train.tokens"
wikitext2_valid_path = "datasets\wikitext-2-v1\wikitext-2\wiki.valid.tokens"
wikitext2_test_path = "datasets\wikitext-2-v1\wikitext-2\wiki.test.tokens"

enwik9_path = "datasets\enwik9\enwik9"

# load datasets
ptb_char_train = datasets.Dataset(ptb_char_train_path)
ptb_char_valid = datasets.Dataset(ptb_char_valid_path)
ptb_word_train = datasets.Dataset(ptb_word_train_path)
ptb_word_valid = datasets.Dataset(ptb_word_valid_path)
ptb_word_test = datasets.Dataset(ptb_word_test_path)

wikitext2_train = datasets.Dataset(wikitext2_train_path)
wikitext2_valid = datasets.Dataset(wikitext2_valid_path)
wikitext2_test = datasets.Dataset(wikitext2_test_path)

enwik9 = datasets.Dataset(enwik9_path)
train_offset = 9*10**7
valid_offset = train_offset + 5*10**6
test_offset = valid_offset + 5*10**6
enwik9_train = enwik9.dataset[:train_offset] # first 90 million for training
enwik9_valid = enwik9.dataset[train_offset: valid_offset] # 5 million for valid
enwik9_test = enwik9.dataset[valid_offset: test_offset] # 5 million for test


5000000

In [ ]:
class Hyperparameters():
    r = 5
    k = int((90 + 40) / 2) # k < min(m,n)
    hidden_states = 4

class Model(object):
    def __init__(self, params):
        # model parameters
        self.r = params.r
        self.k = params.k
        self.lstm = tf.keras.layers.LSTM(3, stateful=True)

    def mogrify(self, x_init, h_0, q, r):
        x = x_init
        h_prev = h_0
        for i in range(1, self.r+1):
            if i % 2 != 0:
                x = 2 * tf.sigmoid(tf.matmul(q[i-1], h_prev)) * x
                # print("odd %s, %s" % (i, tf.shape(x)))
            else:
                h_prev = 2 * tf.sigmoid(tf.matmul(r[i-1], x)) * h_prev
                # print("even %s, %s" % (i, tf.shape(h_prev)))
                
        #inputs = tf.random.normal([32, 10, 8])
        #lstm_cell(inputs, initial_state=tf.zeros([3, 3]))
        
        return x, h_prev

    def matrix_decomposition(self, m, n):
        # currently not decomposition
        # k < min(m,n)

        q_left = tf.random.normal([m, self.k])
        q_right = tf.random.normal([self.k, n])

        r_left = tf.random.normal([n, self.k])
        r_right = tf.random.normal([self.k, m])

        return tf.matmul(q_left, q_right), tf.matmul(r_left, r_right)

    def __call__(self, x):
        return

In [3]:
d = 100
m = d
n = 70
a = 10

x = tf.random.normal([m, a]) # d x a
h_0 = tf.random.normal([n, a]) # n x a

params = Hyperparameters()
model = Model(params)

q = [] # m x n, where m == d
r = [] # n x m, where m == d, it's just transposed Q
for i in range(model.r):
    q_tmp, r_tmp = model.matrix_decomposition(m, n)
    q.append(q_tmp)
    r.append(r_tmp)

model.mogrify(x, h_0, q, r)

In [77]:

lstm_cell  = tf.keras.layers.LSTM(1, stateful=True)
inputs = tf.random.normal([3, 10, 8])
a = lstm_cell(inputs)


In [78]:
a.shape

TensorShape([3, 1])

In [52]:
states_lstm = lstm_cell.states

In [53]:
states_lstm[0].shape

TensorShape([32, 100])

In [90]:
class Model(object):
    def __init__(self, output_dim, r=5, k=65):
        # model parameters
        self.r = r
        self.k = k
        self.output_dim = output_dim
        self.lstm = tf.keras.layers.LSTM(self.output_dim, stateful=True, return_state=True)

    def forward(self, x, state = None):
        
        x, h_prev = self.mogrify(x, )
        
        res = self.lstm(x, initial_state=None)
        
        return res

    def mogrify(self, x_init, h_0, q, r):
        x = x_init
        h_prev = h_0
        for i in range(1, self.r+1):
            if i % 2 != 0:
                x = 2 * tf.sigmoid(tf.matmul(q[i-1], h_prev)) * x
                # print("odd %s, %s" % (i, tf.shape(x)))
            else:
                h_prev = 2 * tf.sigmoid(tf.matmul(r[i-1], x)) * h_prev
                # print("even %s, %s" % (i, tf.shape(h_prev)))
                
        #inputs = tf.random.normal([32, 10, 8])
        #lstm_cell(inputs, initial_state=tf.zeros([3, 3]))
        
        return x, h_prev

    def matrix_decomposition(self, m, n):
        # currently not decomposition
        # k < min(m,n)

        q_left = tf.random.normal([m, self.k])
        q_right = tf.random.normal([self.k, n])

        r_left = tf.random.normal([n, self.k])
        r_right = tf.random.normal([self.k, m])

        return tf.matmul(q_left, q_right), tf.matmul(r_left, r_right)
    #def __call__(self, x):
    #    return

In [128]:
inputs = tf.random.normal([32, 10, 8])
outputs = tf.random.normal([32,1])
lstm_model = Model(1)
a = lstm_model.forward(inputs)

In [129]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [130]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(tf.subtract(target_y, predicted_y)))

In [131]:
@tf.function
def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(outputs, model.forward(inputs))
    grads = t.gradient(current_loss, model.lstm.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.lstm.trainable_variables))


In [132]:
#train_step(inputs, [0,1,0])
loss(lstm_model.forward(inputs)[0],lstm_model.forward(inputs)[2])

<tf.Tensor: shape=(), dtype=float32, numpy=0.18747838>

In [133]:
train(lstm_model, inputs, outputs, 0.001)

In [126]:
lstm_model.forward(inputs)[0]

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-0.02285828],
       [-0.15083903],
       [-0.34129572],
       [-0.15409048],
       [ 0.2064351 ],
       [ 0.09367777],
       [-0.15006217],
       [-0.1653357 ],
       [-0.12802078],
       [-0.004516  ],
       [-0.02686472],
       [ 0.00336589],
       [-0.12070271],
       [-0.15902609],
       [-0.24169014],
       [ 0.02023081],
       [-0.22884719],
       [-0.19811633],
       [-0.06428994],
       [ 0.06493499],
       [ 0.2514327 ],
       [-0.10866537],
       [ 0.00651824],
       [ 0.41850758],
       [-0.10629941],
       [ 0.13327403],
       [ 0.18763158],
       [ 0.11388085],
       [ 0.10960069],
       [ 0.01668897],
       [ 0.3864037 ],
       [-0.00433558]], dtype=float32)>

In [127]:
inputs

<tf.Tensor: shape=(32, 10, 8), dtype=float32, numpy=
array([[[ 0.64471984,  0.32261553, -0.9018336 , ..., -1.2121781 ,
          1.7112691 , -0.06128946],
        [-0.47612506,  2.8363483 ,  0.6488025 , ...,  0.5639938 ,
         -0.89272785,  1.2869956 ],
        [-1.826292  , -1.5898417 ,  1.720043  , ...,  0.35576206,
          1.6709523 , -1.20416   ],
        ...,
        [ 1.3392175 , -0.30057114, -0.312073  , ...,  2.5556269 ,
          0.43465012,  1.5250355 ],
        [-0.94405586,  1.4114699 , -0.27489978, ..., -0.6357156 ,
          0.08776896, -0.78510475],
        [ 1.333204  , -0.90804017,  1.241809  , ...,  0.68062395,
         -0.2806706 , -1.7576079 ]],

       [[ 0.8873787 , -0.446249  , -0.36244833, ...,  1.2092675 ,
          0.06869007,  1.0626215 ],
        [-1.1851996 ,  0.79778206,  0.57506305, ...,  0.27277756,
          0.19615012, -1.2605242 ],
        [-0.15114455,  0.35635394, -0.91550314, ..., -1.1986965 ,
         -0.28736755, -1.4271721 ],
        ...,
 